In [2]:
from ultralytics import YOLO

In [3]:
# Load best run of model
model = YOLO("runs/detect/train/weights/best.pt")

In [6]:
# Make a prediction
prediction = model.predict("../data/images/val/006320.png", save = True)


image 1/1 /Users/langtowl/Desktop/adl-mini-project/notebooks/../data/images/val/006320.png: 224x640 4 Cars, 28.7ms
Speed: 2.2ms preprocess, 28.7ms inference, 0.4ms postprocess per image at shape (1, 3, 224, 640)
Results saved to runs/detect/predict2


In [5]:
# Compute model metrics
metrics = model.val()

Ultralytics 8.3.70 🚀 Python-3.12.4 torch-2.6.0 CPU (Apple M1 Pro)


val: Scanning /Users/langtowl/Desktop/adl-mini-project/data/labels/val.cache... 1497 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1497/1497 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 94/94 [01:54<00:00,  1.22s/it]


                   all       1497       7786      0.672       0.47       0.51      0.307
                   Car       1338       5693      0.831      0.757      0.855      0.593
                   Van        421        553       0.53      0.519      0.512      0.332
                 Truck        208        220      0.614      0.695      0.683      0.486
            Pedestrian        349        873      0.776      0.373      0.508      0.229
        Person_sitting         17         49          1          0      0.124     0.0467
               Cyclist        209        292      0.544      0.295       0.37      0.179
                  Tram         71        106      0.407      0.651      0.515      0.279
Speed: 0.1ms preprocess, 65.9ms inference, 0.0ms loss, 0.2ms postprocess per image
Results saved to runs/detect/val5
